In [366]:
import pandas as pd, numpy as np, sqlite3 as sql, datetime as dt, re, time
from dateutil.relativedelta import relativedelta

In [48]:
with sql.connect('../data/interim/companies.db') as con:
        port = pd.read_sql(f"SELECT * FROM daily ORDER BY Date", con=con).drop_duplicates(subset=['Date', 'symbol'])
        recommends = pd.read_sql(f"SELECT Date, symbol, Firm, new_grade, prev_grade, Action from recommendations ORDER BY Date", con=con)
        arts =pd.read_sql("SELECT date, symbol, publisher, pos_sent, neu_sent, neg_sent, comp_sent FROM articles ORDER BY date", con=con)
        crypt_arts = pd.read_sql("SELECT date, symbol, publisher,pos_sent, neu_sent, neg_sent, comp_sent  FROM news_sentiment ORDER BY date", con=con)
        articles = pd.concat([arts, crypt_arts], axis=0, ignore_index=True)
        comments = pd.read_sql(f"SELECT timestamp, channel, symbols, pos_sent, neu_sent, neg_sent, comp_sent from symbol_comments ORDER BY timestamp", con=con)
        comments.loc[:, "symbols"] = comments.symbols.apply(lambda x: x.replace('BTC', 'BTC-USD'))
        companies = tuple(port.symbol.unique())
        c_data = pd.read_sql(f"SELECT * from mentions WHERE symbol IN {companies}", con=con, index_col='pk')

In [487]:
with sql.connect('../data/interim/companies.db') as con:
    port = pd.read_sql(f"SELECT * FROM daily ORDER BY Date", con=con).drop_duplicates(subset=['Date', 'symbol'])

In [49]:
symbols_re = re.compile(r"\[|\]|\'|\'")
last_index = comments.index.max()

In [50]:
# decompose for single symbol
for i, row in comments.iterrows():
    symbols = re.sub(symbols_re, "", row.symbols)
    symbols = symbols.split(',')
    for sym in symbols:
        last_index+=1
        comments.loc[last_index, ["symbols"]] = sym
        comments.loc[last_index, ["comment_index"]] = i
        comments.loc[last_index, ["timestamp", "channel", "pos_sent", "neu_sent", "neg_sent", "comp_sent"]] = row.timestamp, row.channel, row.pos_sent, row.neu_sent,  row.neg_sent, row.comp_sent


In [51]:
comments = comments[lambda x:~( x.comment_index.isnull())]

In [52]:
comments.loc[comments.index, "timestamp"] = comments.timestamp.apply(pd.to_datetime)

In [53]:
comments

,timestamp,channel,symbols,pos_sent,neu_sent,neg_sent,comp_sent,comment_index
3394,2019-08-08 13:33:00.968000+00:00,wetlqd-ideas,AMD,0.293,0.707,0.000,0.4404,0.0
3395,2019-08-13 14:21:58.192000+00:00,wetlqd-ideas,MSFT,0.182,0.818,0.000,0.4404,1.0
3396,2019-08-13 19:10:23.885000+00:00,wetlqd-ideas,MSFT,0.294,0.706,0.000,0.3612,2.0
3397,2019-08-14 16:35:21.774000+00:00,wetlqd-ideas,ON,0.000,0.597,0.403,-0.4019,3.0
3398,2019-08-14 16:35:21.774000+00:00,wetlqd-ideas,MA,0.000,0.597,0.403,-0.4019,3.0
...,...,...,...,...,...,...,...,...
7660,2020-12-08 20:21:18.936000+00:00,option-trading,OOOO,0.000,1.000,0.000,0.0000,3389.0
7661,2020-12-08 20:21:20.986000+00:00,option-trading,ING,0.900,0.100,0.000,0.8402,3390.0
7662,2020-12-08 20:21:42.264000+00:00,trading,FSLY,0.000,1.000,0.000,0.0000,3391.0
7663,2020-12-08 20:25:14.542000+00:00,trading,IT,0.000,1.000,0.000,0.0000,3392.0


In [56]:
articles.loc[articles.index, "date"] = articles.date.apply(pd.to_datetime)

In [61]:
recommendsDict = {"Very Bearish": 1, "Bearish": 2, "Neutral": 3, "Bullish": 4, "Very Bullish": 5}

In [68]:
recommends=recommends.assign(new_sent = lambda x: x.new_grade.apply(lambda g: recommendsDict[g]))\
    .assign(prev_sent = lambda x: x.prev_grade.apply(lambda g: recommendsDict[g])).assign(Date = lambda x: x.Date.apply(pd.to_datetime)).rename(columns={'Date': 'date'})

In [70]:
comments = comments.rename(columns={"timestamp": "date"}).assign(comment_index=lambda x: x.comment_index.apply(int))

In [90]:
comments = comments[lambda x: x.symbols.isin(companies)]

In [72]:
recommends

,date,symbol,Firm,new_grade,prev_grade,Action,new_sent,prev_sent
0,2012-01-18 00:00:00,BA,Societe Generale,Very Bearish,Bearish,down,1,2
1,2012-02-07 09:24:00,AMD,Longbow Research,Neutral,Very Bullish,up,3,5
2,2012-02-08 00:00:00,DIS,UBS,Neutral,Neutral,main,3,3
3,2012-02-08 00:00:00,DIS,Morgan Stanley,Bullish,Bullish,main,4,4
4,2012-02-09 08:17:00,AAPL,Canaccord Genuity,Very Bullish,Very Bullish,main,5,5
...,...,...,...,...,...,...,...,...
9363,2022-01-06 11:01:33,GILD,Morgan Stanley,Neutral,Bullish,down,3,4
9364,2022-01-11 09:30:24,AMD,Keybanc,Bullish,Neutral,up,4,3
9365,2022-01-11 11:07:38,JD,Atlantic Equities,Bullish,Bullish,init,4,4
9366,2022-01-12 11:54:25,AMD,Barclays,Bullish,Bullish,main,4,4


In [73]:
articles

,date,symbol,publisher,pos_sent,neu_sent,neg_sent,comp_sent
0,2018-01-25,WWR,Seeking Alpha,0.072,0.808,0.120,-0.5719
1,2018-06-27,CBAT,Seeking Alpha,0.078,0.794,0.129,-0.5719
2,2018-08-08,CBAT,Investing.com,0.067,0.858,0.075,-0.6908
3,2018-10-23,XERS,Seeking Alpha,0.127,0.795,0.078,0.4588
4,2018-10-29,SOLO,Investing.com,0.050,0.877,0.073,-0.9100
...,...,...,...,...,...,...,...
26805,2021-09-01,TSLA,Seeking Alpha,0.054,0.908,0.038,0.4767
26806,2021-09-01,TSLA,Seeking Alpha,0.115,0.873,0.012,0.9545
26807,2021-09-01,WKHS,Seeking Alpha,0.110,0.870,0.021,0.9638
26808,2021-09-01,XERS,Seeking Alpha,0.086,0.914,0.000,0.8591


In [109]:
articles_agg = articles.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol']).agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'News')

In [108]:
comments_agg = comments.groupby([pd.Grouper(key="date", freq="1Y"), 'symbols']).agg({'pos_sent': ['mean'], 'neg_sent': ['mean'], 'neu_sent': ['mean'], 'comp_sent': ['mean', 'count']}).assign(type=lambda x: 'Chats')

In [146]:
recommends_agg = recommends.groupby([pd.Grouper(key="date", freq="1Y"), 'symbol']).agg({'new_sent': ['mean'], 'prev_sent': ['mean', 'count']}).assign(type=lambda x: 'Analysts')

In [148]:
recommends_agg.columns = recommends_agg.columns.droplevel(1)

In [151]:
recommends_agg = recommends_agg.reset_index()

In [152]:
comments_agg.columns = comments_agg.columns.droplevel(1)
comments_agg = comments_agg.reset_index()

In [153]:
articles_agg.columns = articles_agg.columns.droplevel(1)
articles_agg = articles_agg.reset_index()

In [158]:
articles_agg.comp_sent.iloc[:, 1].name = "counts"

In [162]:
articles_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']

In [167]:
comments_agg.comp_sent.iloc[:, 1].name = "counts"
comments_agg.columns = ['date', 'symbol', 'pos_sent', 'neg_sent', 'neu_sent', 'comp_sent',
       'counts', 'type']

In [173]:
recommends_agg.prev_sent.iloc[:, 1].name = "counts"
recommends_agg.columns = ['date', 'symbol', 'new_sent', 'prev_sent', 'counts', 'type']

In [194]:
comments_agg=comments_agg.assign(date = lambda x: x.date.apply(lambda x: x.date))

In [488]:
port.loc[port.index, "Date"] = port.Date.apply(pd.to_datetime)

In [489]:
port = port.rename({'Date': 'date'})

In [243]:
comments_agg = comments_agg.assign(date= lambda x: x.date.apply(pd.to_datetime))

In [584]:
# take aggregations over wanted frequency; make buy decisions based off of the frequency of data points and sentiments
# return port with new information: shares and cost * shares
class EAT():
    def __init__(self, portfolio, art_agg, com_agg, rec_agg, start, end):
        self.portfolio = portfolio.copy(deep=True)
        self.postions = []
        self.start = start
        self.end = end
        art_agg = art_agg[lambda x: (x.date >= start) & (x.date <= end)]
        rec_agg = rec_agg[lambda x: (x.date >= start) & (x.date <= end)]
        com_agg = com_agg[lambda x: (x.date >= start) & (x.date <= end)]

        self.aggs = {'articles': art_agg, 'comments': com_agg, 'recommendations': rec_agg}
    
    def tradeSents(self, agg, label, min_samples, min_comp_sent, shares):
        # add action, shares, cost
        returns = self.aggs[agg][lambda x: (x.date >= self.start) & (x[label] >= min_comp_sent) & (x.counts >= min_samples)]
        # query portfolio for first cost add columns
        indexes = pd.Int64Index([])
        for date, sym in returns.loc[:, ['date', 'symbol']].values:
            if sym not in self.postions:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.Date > date) & (x.symbol == sym) & (x.Date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares
            else:
                self.postions.append(sym)
                f1_date = (date + relativedelta(years=1)).to_pydatetime()
                indexes = self.portfolio[lambda x: ((x.Date > date) & (x.symbol == sym) & (x.Date <= f1_date))].index
                self.portfolio.loc[indexes, "shares"] = shares * self.postions.count(sym)
            
            i = returns[lambda x: (x.date == date) & (x.symbol == sym)].index
            if not indexes.empty:
                returns.loc[i, 'cost'] = shares * self.portfolio.loc[indexes[0], "Close"]
                returns.loc[i, 'returns'] = shares * self.portfolio.loc[indexes[-1], "Close"]

        return returns#self.portfolio
        

In [585]:
eat = EAT(port, articles_agg, comments_agg, recommends_agg, dt.datetime(2019, 1, 1), dt.datetime(2022, 1, 4))

In [575]:
#ret = eat.tradeSents("comments", "comp_sent", min_samples=10, min_comp_sent=0.15, shares=10)

In [586]:
ret = eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [494]:
eat.tradeSents("recommendations", "new_sent", 25, 4, 10).value_counts('shares')

shares
10.0    6808
20.0    2887
40.0    2018
30.0    1263
50.0     756
dtype: int64

In [435]:
# eat.tradeSents("articles", "comp_sent", 100, 0.5, 10)
# eat.tradeSents("recommendations", "new_sent", 25, 4, 10)

In [594]:
eat.portfolio[lambda x: x.symbol=='AAPL'

,date,symbol,pos_sent,neg_sent,neu_sent,comp_sent,counts,type,cost,returns
19,2020-12-31,AMD,0.094699,0.034925,0.870377,0.662120,146,News,923.000031,1438.999939
20,2020-12-31,AMZN,0.100262,0.039487,0.860239,0.600201,790,News,31866.298828,33343.400879
23,2020-12-31,BABA,0.080837,0.031374,0.887803,0.617539,436,News,2278.500061,1187.900009
25,2020-12-31,BTC-USD,0.080929,0.033366,0.885643,0.501487,112,News,293741.523438,463064.453125
29,2020-12-31,CRM,0.085547,0.032906,0.881525,0.578460,181,News,2203.099976,2541.300049
32,2020-12-31,DKNG,0.088164,0.023306,0.888516,0.678372,219,News,448.600006,274.699993
41,2020-12-31,HD,0.091755,0.038075,0.870190,0.554736,147,News,2585.193481,4150.100098
46,2020-12-31,INTC,0.076703,0.035745,0.887552,0.530825,239,News,490.681076,515.000000
49,2020-12-31,JD,0.084795,0.027994,0.887263,0.653813,156,News,863.300018,700.699997
57,2020-12-31,MSFT,0.085642,0.038397,0.875970,0.518021,537,News,2166.894226,3363.200073


In [532]:
ret.groupby('date').sum().assign(pct_change = lambda x: ((x.returns - x.cost) / x.cost))

,pos_sent,neg_sent,neu_sent,comp_sent,counts,cost,returns,pct_change
date,,,,,,,,
2019-12-31,0.411192,0.027315,2.561492,0.790218,33,2203.199997,3492.000046,0.584967
2020-12-31,3.708698,0.865937,23.245435,5.331363,1391,327430.133635,500217.853379,0.527709


In [548]:
eat.aggs['articles']

,date,symbol,pos_sent,neg_sent,neu_sent,comp_sent,counts,type
5,2019-12-31,ACB,0.064881,0.056976,0.878190,0.047879,42,News
6,2019-12-31,HEAR,0.151000,0.055000,0.794000,0.991300,1,News
7,2019-12-31,IZEA,0.056000,0.062500,0.881500,-0.310950,2,News
8,2019-12-31,NIO,0.070667,0.049667,0.879667,0.603367,6,News
9,2019-12-31,SOLO,0.054500,0.046000,0.899500,0.117700,2,News
...,...,...,...,...,...,...,...,...
164,2021-12-31,VXX,0.044833,0.038333,0.916833,0.023983,6,News
165,2021-12-31,WKHS,0.079489,0.027748,0.892756,0.598771,131,News
166,2021-12-31,WWR,0.078167,0.010667,0.911250,0.444675,12,News
167,2021-12-31,XERS,0.074633,0.036100,0.889267,0.426207,30,News
